### Redo chat eval and repeat SFT train d20

This notebook is to get back on the GPU machine and do two things from `challenge-30-sft-train-d20/look-into-observations.ipynb`

- I measured chat eval accuracies wrong because the engine didn't yet support calling the python calculator. I want to measure mid and sft again.
  
- I want to measure the validation loss more frequently to help figure out if the increase from step 0 to 100 is concerning.

I'll again use the 8xH100 and again get all the commands ready in this notebook.

Follow the instructions here to get the machine ready: `challenge-28-midtrain-d20/midtrain-d20.ipynb`

In [1]:
!nvidia-smi

Fri Nov 21 19:35:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:61:00.0 Off |                    0 |
| N/A   33C    P0             69W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

#### Do chat eval on mid model

In [3]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=mid --model-tag=d20

W1121 19:35:34.959000 13187 torch/distributed/run.py:803] 
W1121 19:35:34.959000 13187 torch/distributed/run.py:803] *****************************************
W1121 19:35:34.959000 13187 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 19:35:34.959000 13187 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.

#### Do chat eval on sft model

In [5]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20

W1121 19:44:58.832000 16684 torch/distributed/run.py:803] 
W1121 19:44:58.832000 16684 torch/distributed/run.py:803] *****************************************
W1121 19:44:58.832000 16684 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 19:44:58.832000 16684 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

#### Do SFT train with more frequent evaluation of validation loss

In [6]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --eval_every=25 --run=challenge-32-1

W1121 19:51:49.406000 19509 torch/distributed/run.py:803] 
W1121 19:51:49.406000 19509 torch/distributed/run.py:803] *****************************************
W1121 19:51:49.406000 19509 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 19:51:49.406000 19509 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

#### Redo with...

While here, redo with even more frequent evaluation of val loss, more steps for each evaluation, but skip evaluating metrics

In [7]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --eval_every=10 --eval_steps=200 --eval_metrics_every=1000 --run=challenge-32-2

W1121 19:56:39.029000 23301 torch/distributed/run.py:803] 
W1121 19:56:39.029000 23301 torch/distributed/run.py:803] *****************************************
W1121 19:56:39.029000 23301 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 19:56:39.029000 23301 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

#### What happens with an increased target_examples_per_step?

While here try one more thing. Change target_examples_per_step to 64. This will give us 2 grad accum steps. First did:

```
cd /home/ubuntu/mynanochat/chatsft_checkpoints
cp -r d20 d20-teps-32
```

(But realized later unnecessary becuase this new one will have half the number of steps and so get a different filename.)

In [8]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --target_examples_per_step=64 --eval_every=25 --run=challenge-32-3

W1121 20:08:43.589000 25639 torch/distributed/run.py:803] 
W1121 20:08:43.589000 25639 torch/distributed/run.py:803] *****************************************
W1121 20:08:43.589000 25639 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:08:43.589000 25639 torch/distributed/run.py:803] *****************************************
overriding model_tag = d20
overriding target_examples_per_step = 64
overriding eval_every = 25
overriding run = challenge-32-3
user_config: {'run': 'challenge-32-3', 'source': 'mid', 'device_type': '', 'dtype': 'bfloat16', 'device_batch_size': 4, 'num_epochs': 1, 'num_iterations': -1, 'max_data_tokens': -1, 'target_examples_per_step': 64, 'unembedding_lr': 0.004, 'embedding_lr': 0.2, 'matrix_lr': 0.02, 'weight_decay': 0.0, 'init_lr_frac': 0.02, 'eval_every': 25, 'eval_

#### Do chat eval on this sft model

Run chat eval on the model trained just above where target_examples_per_step was 64 leading to ~half the steps.

In [10]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20 --step=349

W1121 20:20:35.302000 30188 torch/distributed/run.py:803] 
W1121 20:20:35.302000 30188 torch/distributed/run.py:803] *****************************************
W1121 20:20:35.302000 30188 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:20:35.302000 30188 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

### Do chat eval on the 700 step SFT model

I hate to do because it's so slow, but when I started running the command above without specifying steps by mistake, and interrupted and started again, I thought the ARC-Easy accuracy didn't match what I measured near the top of this notebook and that's concerning.

In [11]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20

W1121 20:28:07.911000 33105 torch/distributed/run.py:803] 
W1121 20:28:07.911000 33105 torch/distributed/run.py:803] *****************************************
W1121 20:28:07.911000 33105 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:28:07.911000 33105 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

^ The accuracies are different. That shouldn't be. Nothing about changing the engine, of changing how often we evaluate the validation loss should change the trained model.

#### Do SFT train one more time with the defaults

In [12]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --run=challenge-32-4

W1121 20:38:39.772000 36279 torch/distributed/run.py:803] 
W1121 20:38:39.772000 36279 torch/distributed/run.py:803] *****************************************
W1121 20:38:39.772000 36279 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:38:39.772000 36279 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

### Do chat eval

And now repeat chat eval on the model just trained.

In [13]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20

W1121 20:41:49.732000 37868 torch/distributed/run.py:803] 
W1121 20:41:49.732000 37868 torch/distributed/run.py:803] *****************************************
W1121 20:41:49.732000 37868 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:41:49.732000 37868 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

There shouldn't be anything non-deterministic in training or eval. Can I get the same result doing a few evals in a row? Try just for ARC-Easy.

In [14]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --task-name=ARC-Easy --model-tag=d20

W1121 20:49:46.581000 40804 torch/distributed/run.py:803] 
W1121 20:49:46.581000 40804 torch/distributed/run.py:803] *****************************************
W1121 20:49:46.581000 40804 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:49:46.581000 40804 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

ok, that matches the eval above, one more time:

In [15]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --task-name=ARC-Easy --model-tag=d20

W1121 20:50:26.188000 41069 torch/distributed/run.py:803] 
W1121 20:50:26.188000 41069 torch/distributed/run.py:803] *****************************************
W1121 20:50:26.188000 41069 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:50:26.188000 41069 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

matches again, so the eval seems deterministic. Let's SFT train the model again with defaults, just like the last training.

In [16]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --run=challenge-32-5

W1121 20:52:00.461000 41423 torch/distributed/run.py:803] 
W1121 20:52:00.461000 41423 torch/distributed/run.py:803] *****************************************
W1121 20:52:00.461000 41423 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:52:00.461000 41423 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

Now repeat chat eval for ARC-Easy, looking for 44.57% accuracy

In [17]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --task-name=ARC-Easy --model-tag=d20

W1121 20:54:45.271000 42973 torch/distributed/run.py:803] 
W1121 20:54:45.271000 42973 torch/distributed/run.py:803] *****************************************
W1121 20:54:45.271000 42973 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 20:54:45.271000 42973 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

No. So the training is not deterministic? Also can see that final val loss is not identical.

Let me get off of the GPU machine and think about this.

### Back on mac

Well, that didn't go as planned!

Was on the machine for 1.51 hours at a cost of $36.13

Lots of stuff to look into now. Will do that in `investigate-runs.ipynb`